In [2]:
import numpy as np
import ray
import pickle


In [3]:
import os
import sys
#sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))
#print(sys.path)
#from polar_code import polar_code
#parent_dir = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
#os.environ["PYTHONPATH"] = parent_dir + ":" + os.environ.get("PYTHONPATH", "")
#from polar_code import polar_code

In [4]:
ray.init()

2021-11-11 19:33:40,690	INFO resource_spec.py:234 -- Available RAM
2021-11-11 19:33:40,692	INFO resource_spec.py:235 -- Workers: 28.17 GiB
2021-11-11 19:33:40,692	INFO resource_spec.py:236 -- Objects: 14.09 GiB
2021-11-11 19:33:40,694	INFO resource_spec.py:238 -- To adjust these values, use
2021-11-11 19:33:40,697	INFO resource_spec.py:243 --   ray.init(memory=<bytes>, object_store_memory=<bytes>)
2021-11-11 19:33:41,479	INFO services.py:1172 -- Dashboard URL: http://127.0.0.1:8265


{'node_ip_address': '133.34.30.175',
 'raylet_ip_address': '133.34.30.175',
 'redis_address': '133.34.30.175:6379',
 'object_store_address': '/tmp/ray/session_2021-11-11_19-33-40_662771_38569/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-11-11_19-33-40_662771_38569/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-11-11_19-33-40_662771_38569',
 'metrics_export_port': 64091}

In [5]:
@ray.remote
def output(dumped,EbNodB):
        '''
        #あるSNRで計算結果を出力する関数を作成
        #cd.main_func must input 'EbNodB' and output 1D or 2D 'codeword' and 'EST_codeword'
        '''

        #de-seriallize file
        cd=pickle.loads(dumped)
        #seed値の設定
        np.random.seed()

        #prepare some constants
        MAX_ERR=1
        MAX_ALL=10**5
        count_bitall=np.zeros((1))
        count_biterr=np.zeros((1))
        count_all=np.zeros((1))
        count_err=np.zeros((1))
        

        while count_err<MAX_ERR: #count_all<MAX_ALL エラーの個数を指定するか試行回数を指定するかを選ぶ
       
            information,EST_information=cd.main_func(EbNodB) #matrixでのESTinformationにも対応
            
            if EST_information.ndim==1: #change EST_information to 2D 
                EST_information[:,np.newaxis]
                
            data_len=EST_information.shape[1]
            
            #count変数のベクトルを変更(最初のイテレーション時に実行)
            if len(count_all)!=data_len:
                count_bitall=np.zeros((data_len))
                count_biterr=np.zeros((data_len))
                count_all=np.zeros((data_len))
                count_err=np.zeros((data_len))
            
            for i in range(data_len): #EST_information1列ごとに足し算
                #calculate block error rate
                if np.any(information!=EST_information[:,i]):
                    count_err[i]+=1
                count_all[i]+=1

                #calculate bit error rate 
                count_biterr[i]+=np.sum(information!=EST_information[:,i])
                count_bitall[i]+=len(information)

        return count_err,count_all,count_biterr,count_bitall


In [6]:
class MC():
    def __init__(self):
        super().__init__()
        
        self.TX_antenna=1
        self.RX_antenna=1
        self.MAX_ERR=8
        self.EbNodB_start=-5
        self.EbNodB_end=1
        self.EbNodB_range=np.arange(self.EbNodB_start,self.EbNodB_end,0.5) #0.5dBごとに測定

In [7]:
class MC(MC):
    def monte_carlo(self,K):
        '''
        input:main_func
        -----------
        main_func: must input 'EbNodB' and output 1D 'codeword' and 'EST_codeword'
        -----------
        output:BLER,BER

        '''

        BLER=np.zeros(len(self.EbNodB_range))
        BER=np.zeros(len(self.EbNodB_range))

        print("from"+str(self.EbNodB_start)+"to"+str(self.EbNodB_end))
        
        result_ids=[[] for i in range(len(self.EbNodB_range))]

        for i,EbNodB in enumerate(self.EbNodB_range):
            
            for j in range(self.MAX_ERR):
                #multiprocess    
                result_ids[i].append(output.remote(K,EbNodB))  # 並列演算
                #resultは長さ1のリストの中にBLER,BERの2つのarrayのtupleが入った配列

        for i,EbNodB in enumerate(self.EbNodB_range):

            result=ray.get(result_ids[i])

            count_err=0
            count_all=0
            count_biterr=0
            count_bitall=0
            
            for j in range(self.MAX_ERR):
                tmp1,tmp2,tmp3,tmp4=result[j]
                count_err+=tmp1
                count_all+=tmp2
                count_biterr+=tmp3
                count_bitall+=tmp4

            #count_allの要素数が１でなかったときに対応
            if len(count_all)!=1:
                data_len=len(count_all)
                #BER,BLERを二次元に拡張
                BLER=np.zeros((len(self.EbNodB_range),data_len))
                BLER=np.zeros((len(self.EbNodB_range),data_len))
                
            BLER[i]=count_err/count_all
            BER[i]=count_biterr/count_bitall

            if count_biterr/count_bitall<10**-5:
                print("finish")
                break

            print("\r"+"EbNodB="+str(EbNodB)+",BLER="+str(BLER[i])+",BER="+str(BER[i]),end="")
        return BLER,BER


In [8]:
#毎回書き換える関数その１
class savetxt(coding,_AWGN,MC):

  def savetxt(self,BLER,BER):
    
    data_len=1
    #BLERが二次元のときに対応
    if BLER.ndim==2:
      data_len=BLER.shape[1]
      
    for i in range(data_len):
      new_filename=self.filename
      if BLER.ndim==2:
        new_filename=self.filename+str(i) #ファイル名にイテレーション回数を記入
    
      with open(new_filename,'w') as f:

          #print("#N="+str(self.N),file=f)
          print("#TX_antenna="+str(self.TX_antenna),file=f)
          print("#RX_antenna="+str(self.RX_antenna),file=f)
          print("#modulation_symbol="+str(self.M),file=f)
          #print("#MAX_BLERR="+str(self.MAX_ERR),file=f)
          print("#iteration number="+str(self.L_MAX),file=f)
          print("#EsNodB,BLER,BER",file=f) 
          for i in range(len(self.EbNodB_range)):
              print(str(self.EbNodB_range[i]),str(BLER[i]),str(BER[i]),file=f)

NameError: name 'coding' is not defined

In [9]:
if __name__=="__main__":
    K=[400,800,1000]
    for K in K:
        print("K=",K)
        mc=MC()
        BLER,BER=mc.monte_carlo(K)
        st=savetxt(K)
        st.savetxt(BLER,BER)

K= 400
from-5to1


RayTaskError(TypeError): [36mray::output()[39m (pid=38620, ip=133.34.30.175)
  File "python/ray/_raylet.pyx", line 484, in ray._raylet.execute_task
  File "<ipython-input-5-ea2379604ee5>", line 9, in output
TypeError: a bytes-like object is required, not 'int'